# Optimization-based economic models – an introduction

Main topics

- The toolkit
- Consumer choice – a numerical treatment
- Monopoly

## Optimization in Julia

- Julia has a number of packages that solve various classes of optimization problems
    + [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) (more generally, see [JuliaNLSolvers](https://github.com/JuliaNLSolvers))
    + [Optimization.jl](https://github.com/SciML/Optimization.jl)
    + [BlackBoxOptim.jl](https://github.com/robertfeldt/BlackBoxOptim.jl)
- Some of these packages offer different and complementary approaches to the same type of problem
- The diversity of packages results to a proliferation of different APIs for defining and solving problems $\Rightarrow$ there is a need for a unified interface that facilitates access to the different packages
- [JuMP](https://jump.dev/) offers a domain-specific language for defining and working with optimization problems

To install JuMP and a couple of solvers, run the following:

In [ ]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Ipopt")
# Pkg.add("HiGHS") # optional

$$ \min_{x_1,x_2} ~ (x_1-3)^2 + (x_2-2)^2 $$
s.t.
$$x_1^2+x_2^2 \leq 5$$
$$ x_1+x_2 \leq 3 $$
$$ x_1,x_2 \geq 0 $$


In [ ]:
using JuMP, Ipopt

In [ ]:
model = Model(Ipopt.Optimizer)

In [ ]:
@variable(model, x₁ >= 0)

In [ ]:
@variable(model, x₂ >= 0)

In [ ]:
@objective(model,Min,(x₁-3)^2+(x₂-2)^2)

In [ ]:
@constraint(model, c₁, x₁^2 + x₂^2 <= 5)

In [ ]:
@constraint(model, c₂,x₁ + x₂ <= 3)

In [ ]:
print(model)

In [ ]:
optimize!(model)

The output is quite detailed in this case but in principle it is good practice to check if model is solved and a solution found:

In [ ]:
is_solved_and_feasible(model)

In [ ]:
termination_status(model)

`termination_status` provides the reason the solver stopped, i.e. did it succeed, fail etc. [List of termination statuses](https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.TerminationStatusCode)

In [ ]:
primal_status(model)

In [ ]:
dual_status(model)

`primal_status` and `dual_status` tell us how to interpret the result vector. [List of result statuses](https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.ResultStatusCode)

In [ ]:
value(x₁)

In [ ]:
value(x₂)

In [ ]:
shadow_price(c₁)

In [ ]:
shadow_price(c₂)

## Consumer choice – a numerical treatment

Main problem:

$$ \max_{x_1,x_2} x_1^{\alpha_1} x_2^{\alpha_2} $$

s.t.

$$ p_1 x_1 + p_2 x_2 = I $$



In [ ]:
p = [2,2]
I = 20
a1 = 0.4
a2 = 1-a1
umod = Model(Ipopt.Optimizer)
set_silent(umod)
@variable(umod, x[1:2] >= 0)
@constraint(umod, budget, p'*x == I)
@objective(umod, Max, x[1]^a1*x[2]^a2)
optimize!(umod)
cons1, cons2 = value.(x);
println("Consumption of good 1 = $cons1, Consumption of good 2 = $cons2")

#### Plot budget constraint and indifference curve for optimal solution

In [ ]:
using Plots, LaTeXStrings

graph_scale = 1.3

u_star = cons1^a1 * cons2^a2

# Budget line
x1_vals = range(0, I/p[1], length=200)
budget_line = [(I - p[1]*x1)/p[2] for x1 in x1_vals]

# Indifference curve
indiff_x1 = range(0.01, maximum(x1_vals), length=200)  # avoid zero to prevent div-by-zero
indiff_x2 = [(u_star / x1^a1)^(1/a2) for x1 in indiff_x1]


plot(x1_vals, budget_line, label="Budget Line", lw=2, color=:blue)
plot!(indiff_x1, indiff_x2, label="Indifference Curve", lw=2, color=:red)
scatter!([cons1], [cons2], label="Optimal Bundle", color=:black, markersize=6)
xlims!(0, maximum(x1_vals)*graph_scale)
ylims!(0, maximum(budget_line)*graph_scale)
xlabel!(L"$x_1$")
ylabel!(L"$x_2$")

#### The effects of a reduction in $ p_1 $

In [ ]:
p_new = [1, 2]

# New problem
umod2 = Model(Ipopt.Optimizer)
set_silent(umod2)
@variable(umod2, x_new[1:2] >= 0)
@constraint(umod2, p_new' * x_new == I)
@objective(umod2, Max, x_new[1]^a1 * x_new[2]^a2)
optimize!(umod2)
new_cons1, new_cons2 = value.(x_new)
u_new = new_cons1^a1 * new_cons2^a2

# New budget line
x1_vals_new = range(0, I/p_new[1], length=200)
budget_line_new = [(I - p_new[1]*x1)/p_new[2] for x1 in x1_vals_new]

# New indifference curve
indiff_x1_new = range(0.01, maximum(x1_vals_new), length=200)
indiff_x2_new = [(u_new / x1^a1)^(1/a2) for x1 in indiff_x1_new]


plot!(x1_vals_new, budget_line_new, label="New Budget Line", lw=2, color=:blue, linestyle=:dash)
plot!(indiff_x1_new, indiff_x2_new, label="New Indifference Curve", lw=2, color=:red, linestyle=:dash)
scatter!([new_cons1], [new_cons2], label="New Optimal Bundle", color=:black, markershape=:star5, markersize=8)
xlims!(0, maximum(x1_vals_new)*graph_scale)
ylims!(0, maximum(budget_line_new)*graph_scale)


#### Income and substitution effects

In [ ]:
# Illustrate income and substitution effects
umod3 = Model(Ipopt.Optimizer)
set_silent(umod3)

@variable(umod3, x_subs[1:2] >= 0)
@variable(umod3, I_new >= 0)  # flexible income/budget

@constraint(umod3, p' * x_subs == I_new)  # Budget constraint: old prices, variable income
@constraint(umod3, x_subs[1]^a1 * x_subs[2]^a2 == u_new)  # Achieve new utility level as calculated above

@objective(umod3, Min, I_new)  # Minimize budget to reach u_new
optimize!(umod3)

subs_cons1, subs_cons2 = value.(x_subs)
I_val = value(I_new)

# Compensated budget line (same slope as original, passes through compensated bundle)
comp_x1_vals = range(0, I_val / p[1], length=200)
budget_line_comp = [(I_val - p[1]*x1)/p[2] for x1 in comp_x1_vals]

# Plot compensated budget line and bundle
plot!(comp_x1_vals, budget_line_comp, label="Compensated Budget Line", lw=2, color=:green, linestyle=:dash)
scatter!([subs_cons1], [subs_cons2], label="Compensated Bundle", color=:green, markersize=6)


## Monopoly

- When only one firm is operating in a given market, it does not take the price as given. Instead, the monopolist now faces the market demand, as formalized by a market demand function (market demand curve).
- A standard market **demand function** specifies the total quantity $q$ of the respective good that market participants are willing to buy at a given price $p$, i.e. $q(p)$.
- It is sometimes more convenient to work with the **inverse demand function**, denoted by $p(q)$, which specifies the price required for market participants to be willing to buy a given quantity $q$. This is normally a decreasing function of $q$.
- The monopolist's profit is defined in the standard way:
$$ \Pi = R(q) - C(q), $$
where $R(q)$ denotes total revenue and $C(q)$ denotes total cost (an increasing function of $q$).
- Unlike the case of perfect competition, where $R(q) = pq$, here we have
$$ R(q) = p(q)q .$$
- Thus, profit here takes the form
$$ \Pi = p(q)q - C(q) \qquad \qquad \qquad (\text{M-1})$$
- The monopolist wants to maximize profit. Whether this maximum exists and whether it is positive depends on $p(q)$ and $C(q)$.
- Assuming differentiability, the standard optimality condition takes the form
$$ p'(q)q+p(q) = C'(q), \qquad \qquad (\text{M-2})$$
i.e. marginal revenue equals marginal cost.
- Defining the elasticity of a differentiable function $f(x)$ as $\epsilon_f(x) := \frac{f'(x)x}{f(x)}$, we can write equation $(\text{M-2})$ as
$$ p(q)\left[ 1 + \epsilon_p(q) \right] = C'(q) \qquad \qquad (\text{M-3})$$
- Equation $(\text{M-3})$ uses the (quantity) elasticity $\epsilon_p(q)$ of the inverse demand function. It is sometimes more convenient to express it in terms of the (price) elasticity $\epsilon_q(p)$ of the original market demand function $q(p)$, using the fact that $\epsilon_p(q) = 1/\epsilon_q(q)$:
$$ p(q)\left[ 1 + \frac{1}{\epsilon_q(q)} \right] = C'(q) \quad \Leftrightarrow \quad  p(q)\left[ 1 - \frac{1}{|\epsilon_q(q)|} \right] = C'(q).$$
Note that both elasticities are negative in the typical case of downward-sloping demand functions.
- Using the last condition, we can conclude that the optimal quantity cannot be a point where $|\epsilon_q| \leq 1$, since marginal revenue is non-positive and cannot equal (the positive) marginal cost. At a point where the elasticity is less than 1 the monopolist is in a situation where decreasing the quantity produced will both increase revenue and decrease cost, so it cannot be an optimum.
- If $|\epsilon_q| > 1$, for the optimal quantity $q^*$ we can write
$$ p(q^*) = \frac{1}{1-1/|\epsilon_q(q^*)|}C'(q^*) .$$
- Since $\frac{1}{1-1/|\epsilon_q(q^*)|} > 1$, the last expression may be interpreted as saying that price is formed as a *markup* over marginal cost.